Take a look at the [repository](https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix) for more information

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from IPython import display
import time
import os

# proxyの設定．
# keras.datasetsでは，datasetを直接ダウンロードするので，学内マシンからは通常必要．
#os.environ["http_proxy"] = "http://proxy.uec.ac.jp:8080/"
#os.environ["https_proxy"] = "http://proxy.uec.ac.jp:8080/"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"      # "0":GPU0, "1":GPU1, "0,1":GPUを2つとも使用

import numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import math

# Install

In [0]:
!git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix

In [2]:
import os
os.chdir('pytorch-CycleGAN-and-pix2pix/')

In [0]:
!pip install -r requirements.txt

# Datasets

Download one of the official datasets with:

-   `bash ./datasets/download_cyclegan_dataset.sh [apple2orange, orange2apple, summer2winter_yosemite, winter2summer_yosemite, horse2zebra, zebra2horse, monet2photo, style_monet, style_cezanne, style_ukiyoe, style_vangogh, sat2map, map2sat, cityscapes_photo2label, cityscapes_label2photo, facades_photo2label, facades_label2photo, iphone2dslr_flower]`

Or use your own dataset by creating the appropriate folders and adding in the images.

-   Create a dataset folder under `/dataset` for your dataset.
-   Create subfolders `testA`, `testB`, `trainA`, and `trainB` under your dataset's folder. Place any images you want to transform from a to b (cat2dog) in the `testA` folder, images you want to transform from b to a (dog2cat) in the `testB` folder, and do the same for the `trainA` and `trainB` folders.

In [6]:
!bash ./datasets/download_cyclegan_dataset.sh horse2zebra

Specified [horse2zebra]
for details.

--2020-05-10 12:32:14--  https://people.eecs.berkeley.edu/~taesung_park/CycleGAN/datasets/horse2zebra.zip
Resolving proxy.uec.ac.jp (proxy.uec.ac.jp)... 130.153.8.24
Connecting to proxy.uec.ac.jp (proxy.uec.ac.jp)|130.153.8.24|:8080... connected.
Proxy request sent, awaiting response... 200 OK
Length: 116867962 (111M) [application/zip]
Saving to: './datasets/horse2zebra.zip'

100%[======================================>] 116,867,962 26.9MB/s   in 4.9s   

2020-05-10 12:32:19 (22.7 MB/s) - './datasets/horse2zebra.zip' saved [116867962/116867962]

Archive:  ./datasets/horse2zebra.zip
   creating: ./datasets/horse2zebra/trainA/
  inflating: ./datasets/horse2zebra/trainA/n02381460_6223.jpg  
  inflating: ./datasets/horse2zebra/trainA/n02381460_1567.jpg  
  inflating: ./datasets/horse2zebra/trainA/n02381460_3354.jpg  
  inflating: ./datasets/horse2zebra/trainA/n02381460_299.jpg  
  inflating: ./datasets/horse2zebra/trainA/n02381460_3001.jpg  
  inflatin

  inflating: ./datasets/horse2zebra/trainA/n02381460_1122.jpg  
  inflating: ./datasets/horse2zebra/trainA/n02381460_3175.jpg  
  inflating: ./datasets/horse2zebra/trainA/n02381460_4374.jpg  
  inflating: ./datasets/horse2zebra/trainA/n02381460_8639.jpg  
  inflating: ./datasets/horse2zebra/trainA/n02381460_528.jpg  
  inflating: ./datasets/horse2zebra/trainA/n02381460_11.jpg  
  inflating: ./datasets/horse2zebra/trainA/n02381460_3984.jpg  
  inflating: ./datasets/horse2zebra/trainA/n02381460_1135.jpg  
  inflating: ./datasets/horse2zebra/trainA/n02381460_3902.jpg  
  inflating: ./datasets/horse2zebra/trainA/n02381460_199.jpg  
  inflating: ./datasets/horse2zebra/trainA/n02381460_2967.jpg  
  inflating: ./datasets/horse2zebra/trainA/n02381460_677.jpg  
  inflating: ./datasets/horse2zebra/trainA/n02381460_2043.jpg  
  inflating: ./datasets/horse2zebra/trainA/n02381460_1337.jpg  
  inflating: ./datasets/horse2zebra/trainA/n02381460_1769.jpg  
  inflating: ./datasets/horse2zebra/trainA/n0

  inflating: ./datasets/horse2zebra/trainA/n02381460_4491.jpg  
  inflating: ./datasets/horse2zebra/trainA/n02381460_5036.jpg  
  inflating: ./datasets/horse2zebra/trainA/n02381460_503.jpg  
  inflating: ./datasets/horse2zebra/trainA/n02381460_8895.jpg  
  inflating: ./datasets/horse2zebra/trainA/n02381460_1023.jpg  
  inflating: ./datasets/horse2zebra/trainA/n02381460_8157.jpg  
  inflating: ./datasets/horse2zebra/trainA/n02381460_4453.jpg  
  inflating: ./datasets/horse2zebra/trainA/n02381460_1262.jpg  
  inflating: ./datasets/horse2zebra/trainA/n02381460_2402.jpg  
  inflating: ./datasets/horse2zebra/trainA/n02381460_2179.jpg  
  inflating: ./datasets/horse2zebra/trainA/n02381460_1034.jpg  
  inflating: ./datasets/horse2zebra/trainA/n02381460_4417.jpg  
  inflating: ./datasets/horse2zebra/trainA/n02381460_911.jpg  
  inflating: ./datasets/horse2zebra/trainA/n02381460_6556.jpg  
  inflating: ./datasets/horse2zebra/trainA/n02381460_1038.jpg  
  inflating: ./datasets/horse2zebra/trainA

  inflating: ./datasets/horse2zebra/trainA/n02381460_3509.jpg  
  inflating: ./datasets/horse2zebra/trainA/n02381460_631.jpg  
  inflating: ./datasets/horse2zebra/trainA/n02381460_196.jpg  
  inflating: ./datasets/horse2zebra/trainA/n02381460_7597.jpg  
  inflating: ./datasets/horse2zebra/trainA/n02381460_1713.jpg  
  inflating: ./datasets/horse2zebra/trainA/n02381460_2109.jpg  
  inflating: ./datasets/horse2zebra/trainA/n02381460_2317.jpg  
  inflating: ./datasets/horse2zebra/trainA/n02381460_2753.jpg  
  inflating: ./datasets/horse2zebra/trainA/n02381460_4515.jpg  
  inflating: ./datasets/horse2zebra/trainA/n02381460_3783.jpg  
  inflating: ./datasets/horse2zebra/trainA/n02381460_1578.jpg  
  inflating: ./datasets/horse2zebra/trainA/n02381460_1978.jpg  
  inflating: ./datasets/horse2zebra/trainA/n02381460_2684.jpg  
  inflating: ./datasets/horse2zebra/trainA/n02381460_1458.jpg  
  inflating: ./datasets/horse2zebra/trainA/n02381460_967.jpg  
  inflating: ./datasets/horse2zebra/trainA/

  inflating: ./datasets/horse2zebra/trainA/n02381460_803.jpg  
  inflating: ./datasets/horse2zebra/trainA/n02381460_1973.jpg  
  inflating: ./datasets/horse2zebra/trainA/n02381460_1525.jpg  
  inflating: ./datasets/horse2zebra/trainA/n02381460_5045.jpg  
  inflating: ./datasets/horse2zebra/trainA/n02381460_4608.jpg  
  inflating: ./datasets/horse2zebra/trainA/n02381460_1703.jpg  
  inflating: ./datasets/horse2zebra/trainA/n02381460_1266.jpg  
  inflating: ./datasets/horse2zebra/trainA/n02381460_3746.jpg  
  inflating: ./datasets/horse2zebra/trainA/n02381460_971.jpg  
  inflating: ./datasets/horse2zebra/trainA/n02381460_4125.jpg  
  inflating: ./datasets/horse2zebra/trainA/n02381460_7493.jpg  
  inflating: ./datasets/horse2zebra/trainA/n02381460_4538.jpg  
  inflating: ./datasets/horse2zebra/trainA/n02381460_8088.jpg  
  inflating: ./datasets/horse2zebra/trainA/n02381460_6978.jpg  
  inflating: ./datasets/horse2zebra/trainA/n02381460_2217.jpg  
  inflating: ./datasets/horse2zebra/trainA

...

  inflating: ./datasets/horse2zebra/trainB/n02391049_1535.jpg  
  inflating: ./datasets/horse2zebra/trainB/n02391049_978.jpg  
  inflating: ./datasets/horse2zebra/trainB/n02391049_2824.jpg  
  inflating: ./datasets/horse2zebra/trainB/n02391049_6216.jpg  
  inflating: ./datasets/horse2zebra/trainB/n02391049_4693.jpg  
  inflating: ./datasets/horse2zebra/trainB/n02391049_6159.jpg  
  inflating: ./datasets/horse2zebra/trainB/n02391049_787.jpg  
  inflating: ./datasets/horse2zebra/trainB/n02391049_2524.jpg  
  inflating: ./datasets/horse2zebra/trainB/n02391049_11041.jpg  
  inflating: ./datasets/horse2zebra/trainB/n02391049_6097.jpg  
  inflating: ./datasets/horse2zebra/trainB/n02391049_2579.jpg  
  inflating: ./datasets/horse2zebra/trainB/n02391049_578.jpg  
  inflating: ./datasets/horse2zebra/trainB/n02391049_4069.jpg  
  inflating: ./datasets/horse2zebra/trainB/n02391049_10047.jpg  
  inflating: ./datasets/horse2zebra/trainB/n02391049_2356.jpg  
  inflating: ./datasets/horse2zebra/train

  inflating: ./datasets/horse2zebra/trainB/n02391049_2911.jpg  
  inflating: ./datasets/horse2zebra/trainB/n02391049_2691.jpg  
  inflating: ./datasets/horse2zebra/trainB/n02391049_3086.jpg  
  inflating: ./datasets/horse2zebra/trainB/n02391049_4189.jpg  
  inflating: ./datasets/horse2zebra/trainB/n02391049_1199.jpg  
  inflating: ./datasets/horse2zebra/trainB/n02391049_10435.jpg  
  inflating: ./datasets/horse2zebra/trainB/n02391049_2386.jpg  
  inflating: ./datasets/horse2zebra/trainB/n02391049_786.jpg  
  inflating: ./datasets/horse2zebra/trainB/n02391049_8994.jpg  
  inflating: ./datasets/horse2zebra/trainB/n02391049_4879.jpg  
  inflating: ./datasets/horse2zebra/trainB/n02391049_4734.jpg  
  inflating: ./datasets/horse2zebra/trainB/n02391049_2372.jpg  
  inflating: ./datasets/horse2zebra/trainB/n02391049_8593.jpg  
  inflating: ./datasets/horse2zebra/trainB/n02391049_7258.jpg  
  inflating: ./datasets/horse2zebra/trainB/n02391049_268.jpg  
  inflating: ./datasets/horse2zebra/train

  inflating: ./datasets/horse2zebra/trainB/n02391049_713.jpg  
  inflating: ./datasets/horse2zebra/trainB/n02391049_2322.jpg  
  inflating: ./datasets/horse2zebra/trainB/n02391049_127.jpg  
  inflating: ./datasets/horse2zebra/trainB/n02391049_6856.jpg  
  inflating: ./datasets/horse2zebra/trainB/n02391049_1378.jpg  
  inflating: ./datasets/horse2zebra/trainB/n02391049_605.jpg  
  inflating: ./datasets/horse2zebra/trainB/n02391049_3317.jpg  
  inflating: ./datasets/horse2zebra/trainB/n02391049_2693.jpg  
  inflating: ./datasets/horse2zebra/trainB/n02391049_1419.jpg  
  inflating: ./datasets/horse2zebra/trainB/n02391049_8855.jpg  
  inflating: ./datasets/horse2zebra/trainB/n02391049_1125.jpg  
  inflating: ./datasets/horse2zebra/trainB/n02391049_25.jpg  
  inflating: ./datasets/horse2zebra/trainB/n02391049_913.jpg  
  inflating: ./datasets/horse2zebra/trainB/n02391049_9058.jpg  
  inflating: ./datasets/horse2zebra/trainB/n02391049_3187.jpg  
  inflating: ./datasets/horse2zebra/trainB/n02

  inflating: ./datasets/horse2zebra/trainB/n02391049_11063.jpg  
  inflating: ./datasets/horse2zebra/trainB/n02391049_671.jpg  
  inflating: ./datasets/horse2zebra/trainB/n02391049_2729.jpg  
  inflating: ./datasets/horse2zebra/trainB/n02391049_9002.jpg  
  inflating: ./datasets/horse2zebra/trainB/n02391049_4922.jpg  
  inflating: ./datasets/horse2zebra/trainB/n02391049_4909.jpg  
  inflating: ./datasets/horse2zebra/trainB/n02391049_122.jpg  
  inflating: ./datasets/horse2zebra/trainB/n02391049_2949.jpg  
  inflating: ./datasets/horse2zebra/trainB/n02391049_2743.jpg  
  inflating: ./datasets/horse2zebra/trainB/n02391049_2704.jpg  
  inflating: ./datasets/horse2zebra/trainB/n02391049_395.jpg  
  inflating: ./datasets/horse2zebra/trainB/n02391049_357.jpg  
  inflating: ./datasets/horse2zebra/trainB/n02391049_2842.jpg  
  inflating: ./datasets/horse2zebra/trainB/n02391049_2757.jpg  
  inflating: ./datasets/horse2zebra/trainB/n02391049_15.jpg  
  inflating: ./datasets/horse2zebra/trainB/n0

  inflating: ./datasets/horse2zebra/testA/n02381460_140.jpg  
  inflating: ./datasets/horse2zebra/testA/n02381460_4470.jpg  
  inflating: ./datasets/horse2zebra/testA/n02381460_7620.jpg  
  inflating: ./datasets/horse2zebra/testA/n02381460_1210.jpg  
  inflating: ./datasets/horse2zebra/testA/n02381460_7660.jpg  
  inflating: ./datasets/horse2zebra/testA/n02381460_4370.jpg  
  inflating: ./datasets/horse2zebra/testA/n02381460_1830.jpg  
  inflating: ./datasets/horse2zebra/testA/n02381460_5940.jpg  
  inflating: ./datasets/horse2zebra/testA/n02381460_3910.jpg  
  inflating: ./datasets/horse2zebra/testA/n02381460_4450.jpg  
  inflating: ./datasets/horse2zebra/testA/n02381460_2940.jpg  
  inflating: ./datasets/horse2zebra/testA/n02381460_2100.jpg  
  inflating: ./datasets/horse2zebra/testA/n02381460_6950.jpg  
  inflating: ./datasets/horse2zebra/testA/n02381460_4530.jpg  
  inflating: ./datasets/horse2zebra/testA/n02381460_1360.jpg  
  inflating: ./datasets/horse2zebra/testA/n02381460_510.

# Pretrained models

Download one of the official pretrained models with:

-   `bash ./scripts/download_cyclegan_model.sh [apple2orange, orange2apple, summer2winter_yosemite, winter2summer_yosemite, horse2zebra, zebra2horse, monet2photo, style_monet, style_cezanne, style_ukiyoe, style_vangogh, sat2map, map2sat, cityscapes_photo2label, cityscapes_label2photo, facades_photo2label, facades_label2photo, iphone2dslr_flower]`

Or add your own pretrained model to `./checkpoints/{NAME}_pretrained/latest_net_G.pt`

In [0]:
!bash ./scripts/download_cyclegan_model.sh horse2zebra

# Training

-   `python train.py --dataroot ./datasets/horse2zebra --name horse2zebra --model cycle_gan`

Change the `--dataroot` and `--name` to your own dataset's path and model's name. Use `--gpu_ids 0,1,..` to train on multiple GPUs and `--batch_size` to change the batch size. I've found that a batch size of 16 fits onto 4 V100s and can finish training an epoch in ~90s.

Once your model has trained, copy over the last checkpoint to a format that the testing model can automatically detect:

Use `cp ./checkpoints/horse2zebra/latest_net_G_A.pth ./checkpoints/horse2zebra/latest_net_G.pth` if you want to transform images from class A to class B and `cp ./checkpoints/horse2zebra/latest_net_G_B.pth ./checkpoints/horse2zebra/latest_net_G.pth` if you want to transform images from class B to class A.


In [3]:
%run train.py --dataroot ./datasets/horse2zebra --name horse2zebra --model cycle_gan --batch_size 8 --gpu_ids 0,1

----------------- Options ---------------
               batch_size: 8                             	[default: 1]
                    beta1: 0.5                           
          checkpoints_dir: ./checkpoints                 
           continue_train: False                         
                crop_size: 256                           
                 dataroot: ./datasets/horse2zebra        	[default: None]
             dataset_mode: unaligned                     
                direction: AtoB                          
              display_env: main                          
             display_freq: 400                           
               display_id: 1                             
            display_ncols: 4                             
             display_port: 8097                          
           display_server: http://localhost              
          display_winsize: 256                           
                    epoch: latest                        
 

Setting up a new session...


create web directory ./checkpoints/horse2zebra/web...
(epoch: 1, iters: 200, time: 0.136, data: 0.185) D_A: 0.627 G_A: 0.644 cycle_A: 3.439 idt_A: 1.507 D_B: 1.003 G_B: 1.416 cycle_B: 3.170 idt_B: 1.561 
(epoch: 1, iters: 400, time: 0.178, data: 0.002) D_A: 0.373 G_A: 0.369 cycle_A: 2.119 idt_A: 1.363 D_B: 0.361 G_B: 0.449 cycle_B: 2.843 idt_B: 1.013 
(epoch: 1, iters: 600, time: 0.147, data: 0.003) D_A: 0.304 G_A: 0.363 cycle_A: 2.728 idt_A: 1.069 D_B: 0.302 G_B: 0.407 cycle_B: 2.331 idt_B: 1.314 
(epoch: 1, iters: 800, time: 0.154, data: 0.002) D_A: 0.288 G_A: 0.358 cycle_A: 2.008 idt_A: 1.079 D_B: 0.255 G_B: 0.385 cycle_B: 2.418 idt_B: 0.923 
(epoch: 1, iters: 1000, time: 0.143, data: 0.003) D_A: 0.278 G_A: 0.294 cycle_A: 2.779 idt_A: 1.242 D_B: 0.276 G_B: 0.320 cycle_B: 3.164 idt_B: 1.171 
(epoch: 1, iters: 1200, time: 0.156, data: 0.002) D_A: 0.270 G_A: 0.352 cycle_A: 1.871 idt_A: 1.037 D_B: 0.314 G_B: 0.389 cycle_B: 2.125 idt_B: 0.785 
End of epoch 1 / 200 	 Time Taken: 188 sec
l

(epoch: 8, iters: 848, time: 0.139, data: 0.002) D_A: 0.170 G_A: 0.459 cycle_A: 2.327 idt_A: 0.790 D_B: 0.229 G_B: 0.437 cycle_B: 1.625 idt_B: 0.949 
(epoch: 8, iters: 1048, time: 0.160, data: 0.002) D_A: 0.203 G_A: 0.594 cycle_A: 1.383 idt_A: 0.699 D_B: 0.163 G_B: 0.313 cycle_B: 1.392 idt_B: 0.674 
(epoch: 8, iters: 1248, time: 0.138, data: 0.003) D_A: 0.147 G_A: 0.446 cycle_A: 1.466 idt_A: 0.733 D_B: 0.215 G_B: 0.304 cycle_B: 1.549 idt_B: 0.579 
End of epoch 8 / 200 	 Time Taken: 189 sec
learning rate = 0.0002000
(epoch: 9, iters: 112, time: 0.188, data: 0.002) D_A: 0.186 G_A: 0.371 cycle_A: 1.386 idt_A: 0.825 D_B: 0.238 G_B: 0.418 cycle_B: 1.850 idt_B: 0.583 
(epoch: 9, iters: 312, time: 0.140, data: 0.003) D_A: 0.153 G_A: 0.387 cycle_A: 1.475 idt_A: 0.733 D_B: 0.219 G_B: 0.431 cycle_B: 1.701 idt_B: 0.608 
(epoch: 9, iters: 512, time: 0.153, data: 0.002) D_A: 0.234 G_A: 0.241 cycle_A: 1.611 idt_A: 0.765 D_B: 0.445 G_B: 0.702 cycle_B: 1.587 idt_B: 0.706 
(epoch: 9, iters: 712, time: 

saving the model at the end of epoch 15, iters 20040
End of epoch 15 / 200 	 Time Taken: 191 sec
learning rate = 0.0002000
(epoch: 16, iters: 160, time: 0.139, data: 0.003) D_A: 0.192 G_A: 0.212 cycle_A: 1.569 idt_A: 0.660 D_B: 0.282 G_B: 0.635 cycle_B: 1.351 idt_B: 0.661 
(epoch: 16, iters: 360, time: 0.186, data: 0.003) D_A: 0.217 G_A: 0.400 cycle_A: 1.487 idt_A: 0.840 D_B: 0.200 G_B: 0.398 cycle_B: 1.740 idt_B: 0.655 
(epoch: 16, iters: 560, time: 0.141, data: 0.003) D_A: 0.147 G_A: 0.384 cycle_A: 1.497 idt_A: 0.625 D_B: 0.241 G_B: 0.515 cycle_B: 1.282 idt_B: 0.717 
(epoch: 16, iters: 760, time: 0.156, data: 0.003) D_A: 0.275 G_A: 0.517 cycle_A: 1.428 idt_A: 0.606 D_B: 0.194 G_B: 0.256 cycle_B: 1.367 idt_B: 0.590 
(epoch: 16, iters: 960, time: 0.140, data: 0.003) D_A: 0.148 G_A: 0.506 cycle_A: 1.501 idt_A: 0.600 D_B: 0.185 G_B: 0.461 cycle_B: 1.367 idt_B: 0.649 
(epoch: 16, iters: 1160, time: 0.162, data: 0.002) D_A: 0.282 G_A: 0.504 cycle_A: 1.330 idt_A: 0.574 D_B: 0.228 G_B: 0.463

...

End of epoch 172 / 200 	 Time Taken: 190 sec
learning rate = 0.0000554
(epoch: 173, iters: 8, time: 0.146, data: 0.002) D_A: 0.090 G_A: 0.528 cycle_A: 0.611 idt_A: 0.247 D_B: 0.142 G_B: 0.553 cycle_B: 0.746 idt_B: 0.219 
(epoch: 173, iters: 208, time: 0.279, data: 0.003) D_A: 0.130 G_A: 0.407 cycle_A: 0.658 idt_A: 0.264 D_B: 0.189 G_B: 0.480 cycle_B: 0.695 idt_B: 0.267 
saving the latest model (epoch 173, total_iters 230000)
(epoch: 173, iters: 408, time: 0.142, data: 0.002) D_A: 0.106 G_A: 0.477 cycle_A: 0.557 idt_A: 0.253 D_B: 0.124 G_B: 0.507 cycle_B: 0.662 idt_B: 0.193 
(epoch: 173, iters: 608, time: 0.159, data: 0.002) D_A: 0.113 G_A: 0.499 cycle_A: 0.662 idt_A: 0.231 D_B: 0.170 G_B: 0.342 cycle_B: 0.629 idt_B: 0.242 
(epoch: 173, iters: 808, time: 0.150, data: 0.003) D_A: 0.108 G_A: 0.491 cycle_A: 0.578 idt_A: 0.240 D_B: 0.130 G_B: 0.568 cycle_B: 0.668 idt_B: 0.197 
(epoch: 173, iters: 1008, time: 0.169, data: 0.003) D_A: 0.177 G_A: 0.403 cycle_A: 0.531 idt_A: 0.285 D_B: 0.191 G_

(epoch: 180, iters: 656, time: 0.146, data: 0.002) D_A: 0.140 G_A: 0.451 cycle_A: 0.540 idt_A: 0.223 D_B: 0.154 G_B: 0.404 cycle_B: 0.549 idt_B: 0.173 
(epoch: 180, iters: 856, time: 0.290, data: 0.002) D_A: 0.158 G_A: 0.595 cycle_A: 0.545 idt_A: 0.217 D_B: 0.124 G_B: 0.508 cycle_B: 0.601 idt_B: 0.197 
saving the latest model (epoch 180, total_iters 240000)
(epoch: 180, iters: 1056, time: 0.140, data: 0.003) D_A: 0.110 G_A: 0.537 cycle_A: 0.534 idt_A: 0.266 D_B: 0.198 G_B: 0.371 cycle_B: 0.696 idt_B: 0.182 
(epoch: 180, iters: 1256, time: 0.154, data: 0.002) D_A: 0.148 G_A: 0.637 cycle_A: 0.537 idt_A: 0.209 D_B: 0.159 G_B: 0.381 cycle_B: 0.578 idt_B: 0.232 
saving the model at the end of epoch 180, iters 240480
End of epoch 180 / 200 	 Time Taken: 195 sec
learning rate = 0.0000396
(epoch: 181, iters: 120, time: 0.142, data: 0.002) D_A: 0.118 G_A: 0.386 cycle_A: 0.611 idt_A: 0.252 D_B: 0.194 G_B: 0.624 cycle_B: 0.705 idt_B: 0.222 
(epoch: 181, iters: 320, time: 0.297, data: 0.002) D_A: 

(epoch: 187, iters: 1304, time: 0.146, data: 0.002) D_A: 0.114 G_A: 0.608 cycle_A: 0.524 idt_A: 0.211 D_B: 0.130 G_B: 0.435 cycle_B: 0.635 idt_B: 0.192 
End of epoch 187 / 200 	 Time Taken: 190 sec
learning rate = 0.0000257
(epoch: 188, iters: 168, time: 0.300, data: 0.002) D_A: 0.107 G_A: 0.464 cycle_A: 0.555 idt_A: 0.249 D_B: 0.201 G_B: 0.397 cycle_B: 0.726 idt_B: 0.190 
saving the latest model (epoch 188, total_iters 250000)
(epoch: 188, iters: 368, time: 0.138, data: 0.008) D_A: 0.138 G_A: 0.503 cycle_A: 0.510 idt_A: 0.249 D_B: 0.202 G_B: 0.471 cycle_B: 0.718 idt_B: 0.183 
(epoch: 188, iters: 568, time: 0.154, data: 0.002) D_A: 0.085 G_A: 0.574 cycle_A: 0.499 idt_A: 0.195 D_B: 0.159 G_B: 0.518 cycle_B: 0.564 idt_B: 0.181 
(epoch: 188, iters: 768, time: 0.146, data: 0.002) D_A: 0.157 G_A: 0.436 cycle_A: 0.498 idt_A: 0.211 D_B: 0.204 G_B: 0.493 cycle_B: 0.599 idt_B: 0.183 
(epoch: 188, iters: 968, time: 0.165, data: 0.004) D_A: 0.150 G_A: 0.452 cycle_A: 0.529 idt_A: 0.177 D_B: 0.197 

(epoch: 195, iters: 616, time: 0.145, data: 0.002) D_A: 0.127 G_A: 0.555 cycle_A: 0.553 idt_A: 0.274 D_B: 0.161 G_B: 0.411 cycle_B: 0.658 idt_B: 0.184 
(epoch: 195, iters: 816, time: 0.300, data: 0.002) D_A: 0.159 G_A: 0.439 cycle_A: 0.547 idt_A: 0.202 D_B: 0.164 G_B: 0.483 cycle_B: 0.567 idt_B: 0.217 
saving the latest model (epoch 195, total_iters 260000)
(epoch: 195, iters: 1016, time: 0.151, data: 0.005) D_A: 0.113 G_A: 0.557 cycle_A: 0.493 idt_A: 0.195 D_B: 0.139 G_B: 0.422 cycle_B: 0.595 idt_B: 0.176 
(epoch: 195, iters: 1216, time: 0.156, data: 0.004) D_A: 0.135 G_A: 0.624 cycle_A: 0.552 idt_A: 0.242 D_B: 0.132 G_B: 0.463 cycle_B: 0.609 idt_B: 0.203 
saving the model at the end of epoch 195, iters 260520
End of epoch 195 / 200 	 Time Taken: 195 sec
learning rate = 0.0000099
(epoch: 196, iters: 80, time: 0.141, data: 0.003) D_A: 0.133 G_A: 0.512 cycle_A: 0.574 idt_A: 0.211 D_B: 0.140 G_B: 0.442 cycle_B: 0.575 idt_B: 0.204 
(epoch: 196, iters: 280, time: 0.297, data: 0.002) D_A: 0

KeyboardInterrupt: 

# Testing

-   `python test.py --dataroot datasets/horse2zebra/testA --name horse2zebra_pretrained --model test --no_dropout`

Change the `--dataroot` and `--name` to be consistent with your trained model's configuration.

> from https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix:
> The option --model test is used for generating results of CycleGAN only for one side. This option will automatically set --dataset_mode single, which only loads the images from one set. On the contrary, using --model cycle_gan requires loading and generating results in both directions, which is sometimes unnecessary. The results will be saved at ./results/. Use --results_dir {directory_path_to_save_result} to specify the results directory.

> For your own experiments, you might want to specify --netG, --norm, --no_dropout to match the generator architecture of the trained model.

In [8]:
%run test.py --dataroot datasets/horse2zebra --name horse2zebra --model cycle_gan --no_dropout

----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: ./checkpoints                 
                crop_size: 256                           
                 dataroot: datasets/horse2zebra          	[default: None]
             dataset_mode: unaligned                     
                direction: AtoB                          
          display_winsize: 256                           
                    epoch: latest                        
                     eval: False                         
                  gpu_ids: 0                             
                init_gain: 0.02                          
                init_type: normal                        
                 input_nc: 3                             
                  isTrain: False                         	[default: None]
                load_iter: 0                            

# Visualize

In [ ]:
import matplotlib.pyplot as plt

img = plt.imread('./results/horse2zebra/test_latest/images/n02381460_1010_fake.png')
plt.imshow(img)

In [0]:
import matplotlib.pyplot as plt

img = plt.imread('./results/horse2zebra_pretrained/test_latest/images/n02381460_1010_real.png')
plt.imshow(img)